# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [17]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://www.myjoyonline.com/")
huggingface.links

['/',
 '/',
 'https://www.myjoyonline.com/news/',
 'https://www.myjoyonline.com/arts-culture/',
 'https://www.myjoyonline.com/business/',
 'https://www.myjoyonline.com/sports/',
 'https://www.myjoyonline.com/opinion/',
 'https://www.myjoyonline.com/research/',
 '/live',
 'https://www.myjoyonline.com/media/',
 'https://myjoyonline.com/election',
 '/',
 '/',
 'https://www.myjoyonline.com/news/',
 'https://www.myjoyonline.com/arts-culture/',
 'https://www.myjoyonline.com/business/',
 'https://www.myjoyonline.com/sports/',
 'https://www.myjoyonline.com/opinion/',
 'https://www.myjoyonline.com/research/',
 '/live',
 'https://www.myjoyonline.com/media/',
 'https://myjoyonline.com/election',
 'https://www.youtube.com/user/myjoyonlinetube?sub_confirmation=1',
 'https://web.facebook.com/JoyNewsOnTV/',
 'https://twitter.com/Joy997FM',
 '#',
 '/category/top-story',
 'https://www.myjoyonline.com/majority-leader-justifies-2-45-electricity-tariff-hike/',
 'https://www.myjoyonline.com/majority-leader

In [19]:
get_links("https://www.myjoyonline.com/")

{'links': [{'type': 'main page', 'url': 'https://www.myjoyonline.com/'},
  {'type': 'news', 'url': 'https://www.myjoyonline.com/news/'},
  {'type': 'media', 'url': 'https://www.myjoyonline.com/media/'},
  {'type': 'about page', 'url': 'https://www.multimediaghana.com'},
  {'type': 'careers page', 'url': 'https://myjoyonline.com/careers'},
  {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [21]:
print(get_all_details("https://www.myjoyonline.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}, {'type': 'careers page', 'url': 'https://myjoyonline.com/careers'}]}
Landing page:
Webpage Title:
MyJoyOnline - Ghana News | Ghana's most comprehensive website. Independent, Fearless and Credible journalism
Webpage Contents:
Home
News
Arts and Culture
Business
Sports
Opinion
Research
Live TV/Radio
Media
Elections
Dark Mode
Dark mode
Home
News
Arts and Culture
Business
Sports
Opinion
Research
Live TV/Radio
Media
Elections
Forgot Password
Create Account
https://www.myjoyonline.com/-------https://www.myjoyonline.com/
Receive news updates on the go.
Select one or more categories
News
Business
Sports
Opinion
Entertainment
Breaking News
I want to receive news:
Daily
Weekly
Monthly
Daily: 0.30p | Weekly: GHS
                            2 | Monthly: GHs 9
Top story
Majority Leader justifies 2.45% electricity tariff hike
News
Ghana 

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
get_brochure_user_prompt("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'careers page', 'url': 'https://www.myjoyonline.com/contact-us'}, {'type': 'contact page', 'url': 'https://www.myjoyonline.com/contact-us'}]}


"You are looking at a company called: Multimedia Ghana\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nMyJoyOnline - Ghana News | Ghana's most comprehensive website. Independent, Fearless and Credible journalism\nWebpage Contents:\nHome\nNews\nArts and Culture\nBusiness\nSports\nOpinion\nResearch\nLive TV/Radio\nMedia\nElections\nDark Mode\nDark mode\nHome\nNews\nArts and Culture\nBusiness\nSports\nOpinion\nResearch\nLive TV/Radio\nMedia\nElections\nForgot Password\nCreate Account\nhttps://www.myjoyonline.com/-------https://www.myjoyonline.com/\nReceive news updates on the go.\nSelect one or more categories\nNews\nBusiness\nSports\nOpinion\nEntertainment\nBreaking News\nI want to receive news:\nDaily\nWeekly\nMonthly\nDaily: 0.30p | Weekly: GHS\r\n                            2 | Monthly: GHs 9\nTop story\nMajority Leader justifies 2.45% electricity tariff hike

In [25]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'careers page', 'url': 'https://www.myjoyonline.com/contact-us'}, {'type': 'news page', 'url': 'https://www.myjoyonline.com/news/'}, {'type': 'media page', 'url': 'https://www.myjoyonline.com/media/'}, {'type': 'election page', 'url': 'https://myjoyonline.com/election'}]}


# Multimedia Ghana Brochure

---

## Who We Are

**Multimedia Ghana** is a leading media organization in Ghana, committed to delivering independent, fearless, and credible journalism. Through various platforms including **MyJoyOnline**, we provide comprehensive news coverage in areas such as business, sports, arts, culture, and current affairs. Our reach includes radio, television, and online content, making us a pivotal source of information for Ghanaians.

## Our Vision

Our mission is to uphold and promote the values of transparency and accountability in journalism while ensuring informed public discourse. We aim to be the go-to source for relevant and trustworthy news, fostering an informed citizenry in Ghana and beyond.

---

## Our Offerings

We cover a wide array of topics across multiple platforms:

- **News**: Stay updated with breaking news, local stories, and international affairs.
- **Business**: Insightful reporting on the economic landscape, financial markets, and corporate news.
- **Sports**: Coverage of local and international sporting events, debates, and athlete profiles.
- **Arts and Culture**: Engaging content that celebrates Ghanaian heritage and talents.
- **Opinion and Research**: Analytical pieces that encourage public discourse and critical thinking.

### Tune In
- **Live TV & Radio**: Experience our broadcasts with a variety of programs tailored for different audiences, including **Super Morning Show**, **DriveTime**, and more. 

---

## Our Customers

We serve a diverse audience, including:
- General Public: Individuals seeking trustworthy news and entertainment.
- Businesses: Organizations requiring market insights and advertising opportunities.
- Educational Institutions: Collaborations for informative programs and events.

## Company Culture

At Multimedia Ghana, we pride ourselves on fostering a collaborative and innovative work environment. We believe in:

- **Integrity**: Commitment to ethical journalism and transparency.
- **Diversity**: Embracing various perspectives, backgrounds, and ideas within our workforce.
- **Growth**: Encouraging professional development and continuous learning.

---

## Careers with Us

We're always on the lookout for passionate individuals who want to make a difference in the media landscape. Whether you are a seasoned professional or an enthusiastic newcomer, there's a place for you at Multimedia Ghana. Joining our team means engaging in dynamic projects while being part of a mission-driven organization.

### Current Opportunities
- Journalists
- Editors
- Multimedia Producers
- Marketing Specialists
- IT Professionals

Visit our careers page on [MyJoyOnline](https://www.myjoyonline.com) for the latest job openings and application instructions.

---

## Connect With Us

Stay informed and engaged! Follow us on our social media channels and subscribe to our newsletters to receive the latest updates and offerings from Multimedia Ghana.

- **Website**: [MyJoyOnline](https://www.myjoyonline.com)
- **Social Media**: Follow us on Facebook, Twitter, and Instagram!

Join us and be part of a community that values journalism and its power to inform and inspire!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [27]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("Multimedia Ghana", "https://www.myjoyonline.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.multimediaghana.com'}, {'type': 'careers page', 'url': 'https://www.myjoyonline.com/contact-us'}]}


# Multimedia Ghana Brochure

## About Us
Welcome to **Multimedia Ghana**, home of **MyJoyOnline**, the leading and most comprehensive news website in Ghana. We proudly stand as a beacon of independent, fearless, and credible journalism, ensuring that our community stays informed through a variety of channels including news, live TV, radio, and podcasts.

---

## Our Offerings
- **News**: Daily updates covering a wide range of topics from breaking news, business insights, and engaging opinion pieces to sports and lifestyle.
- **Live Broadcasts**: Tune into our live TV and radio segments to stay connected with current affairs and entertainment.
- **Podcasts and Features**: Explore a diverse array of shows such as Super Morning Show, DriveTime, and Joy Business Report for real-time discussions and in-depth analyses.

---

## Our Culture
At Multimedia Ghana, we foster a culture of creativity, integrity, and teamwork. Our environment encourages open dialogue and empowers our team to push the boundaries of storytelling. Here, you will find:
- **Collaboration over Competition**: We believe that teamwork leads to the best outcomes for our audience and our employees.
- **Continuous Learning**: We invest in the professional development of our team, providing opportunities for skills enhancement and growth.
- **Diversity and Inclusion**: Our workplace thrives on diverse perspectives, ensuring that every voice is heard and respected.

---

## Community and Customer Engagement
Our commitment to our audience extends beyond just delivering news; we actively engage with our community through initiatives like:
- **Joy Business**: Connecting businesses with valuable insights to drive growth.
- **Public Debates and Discussions**: Hosting forums to discuss pressing issues that matter to our citizens.

---

## Career Opportunities
We are constantly looking for passionate individuals to join our growing family. At Multimedia Ghana, you will have the opportunity to work in an innovative environment that values your creativity and contributions. Whether in journalism, media production, tech, or marketing, we invite you to explore various career paths within our dynamic team.

Join us and be part of a future that promotes the significant advancements in media and storytelling in Ghana and beyond!

---

## Stay Connected
Visit our website at [www.myjoyonline.com](https://www.myjoyonline.com) for the latest updates, or sign up for personalized news alerts tailored to your interests. 

Let's build a well-informed society together!

--- 

For inquiries about partnerships, advertising, or career opportunities, please contact us at: info@multimediaghana.com.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>